In [ ]:
!pip install transformers datasets scikit-learn imbalanced-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# ***Preprocessing***

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

data = pd.read_csv('bias.csv')

data = data[['docbody', 'article_bias']].dropna()
data.rename(columns={'docbody': 'text', 'article_bias': 'label'}, inplace=True)

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_idx, temp_idx in stratified_split.split(data['text'], data['label']):
    train_texts = data['text'].iloc[train_idx]
    train_labels = data['label'].iloc[train_idx]
    temp_texts = data['text'].iloc[temp_idx]
    temp_labels = data['label'].iloc[temp_idx]

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for val_idx, test_idx in stratified_split.split(temp_texts, temp_labels):
    val_texts = temp_texts.iloc[val_idx]
    val_labels = temp_labels.iloc[val_idx]
    test_texts = temp_texts.iloc[test_idx]
    test_labels = temp_labels.iloc[test_idx]

print("Training set class distribution:")
print(train_labels.value_counts())
print("Validation set class distribution:")
print(val_labels.value_counts())
print("Test set class distribution:")
print(test_labels.value_counts())


Training set class distribution:
label
3    64
2    50
1    23
4    13
Name: count, dtype: int64
Validation set class distribution:
label
3    14
2    11
1     4
4     3
Name: count, dtype: int64
Test set class distribution:
label
3    14
2    11
1     5
4     3
Name: count, dtype: int64


In [ ]:
unique_labels = sorted(train_labels.unique())
label_map = {label: i for i, label in enumerate(unique_labels)}

train_labels = train_labels.map(label_map)
val_labels = val_labels.map(label_map)
test_labels = test_labels.map(label_map)

print(f"Label Mapping: {label_map}")
print("Training set after mapping:")
print(train_labels.value_counts())
print("Validation set after mapping:")
print(val_labels.value_counts())
print("Test set after mapping:")
print(test_labels.value_counts())


Label Mapping: {1: 0, 2: 1, 3: 2, 4: 3}
Training set after mapping:
label
2    64
1    50
0    23
3    13
Name: count, dtype: int64
Validation set after mapping:
label
2    14
1    11
0     4
3     3
Name: count, dtype: int64
Test set after mapping:
label
2    14
1    11
0     5
3     3
Name: count, dtype: int64


In [ ]:
from imblearn.over_sampling import RandomOverSampler

texts = train_texts.tolist()
labels = train_labels.tolist()

ros = RandomOverSampler(random_state=42)
oversampled_texts, oversampled_labels = ros.fit_resample(
    pd.DataFrame({'text': texts}), pd.Series(labels)
)

train_texts = oversampled_texts['text']
train_labels = oversampled_labels

print("Oversampled training set class distribution:")
print(train_labels.value_counts())


Oversampled training set class distribution:
1    64
2    64
0    64
3    64
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict({'text': train_texts.tolist(), 'label': train_labels.tolist()})
val_dataset = Dataset.from_dict({'text': val_texts.tolist(), 'label': val_labels.tolist()})
test_dataset = Dataset.from_dict({'text': test_texts.tolist(), 'label': test_labels.tolist()})


In [ ]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize_function(examples):
    return tokenizer(
        examples['text'], truncation=True, padding='max_length', max_length=512
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

tokenized_train = tokenized_train.map(lambda x: {'label': int(x['label'])})
tokenized_val = tokenized_val.map(lambda x: {'label': int(x['label'])})
tokenized_test = tokenized_test.map(lambda x: {'label': int(x['label'])})

tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

# **Training Parameters**

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

CUDA_LAUNCH_BLOCKING=1

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(label_map)
)

training_args = TrainingArguments(
    output_dir="./bert_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./bert_logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# **Training And Evaluation**

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
CUDA_LAUNCH_BLOCKING=1
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted", zero_division=1),
        "precision": precision_score(labels, predictions, average="weighted", zero_division=1),
        "recall": recall_score(labels, predictions, average="weighted", zero_division=1),
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


<ipython-input-8-77ce35a8179a>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

val_results = trainer.evaluate(eval_dataset=tokenized_val)
print("Validation Set Results:", val_results)

test_results = trainer.evaluate(eval_dataset=tokenized_test)
print("Test Set Results:", test_results)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.441282,0.187500,0.188945,0.579799,0.187500
2,No log,1.457827,0.218750,0.221034,0.390278,0.218750
3,No log,1.487395,0.187500,0.159799,0.156250,0.187500
4,No log,1.556102,0.218750,0.216890,0.377083,0.218750
5,No log,1.566315,0.218750,0.216890,0.377083,0.218750


Validation Set Results: {'eval_loss': 1.457827091217041, 'eval_accuracy': 0.21875, 'eval_f1': 0.22103365384615387, 'eval_precision': 0.3902777777777778, 'eval_recall': 0.21875, 'eval_runtime': 0.989, 'eval_samples_per_second': 32.357, 'eval_steps_per_second': 2.022, 'epoch': 5.0}
Test Set Results: {'eval_loss': 1.4115556478500366, 'eval_accuracy': 0.30303030303030304, 'eval_f1': 0.2680926916221034, 'eval_precision': 0.34595959595959597, 'eval_recall': 0.30303030303030304, 'eval_runtime': 0.9702, 'eval_samples_per_second': 34.013, 'eval_steps_per_second': 3.092, 'epoch': 5.0}


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    warmup_steps=500,
    logging_dir="./bert_logs",
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
    report_to="none",


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

val_results = trainer.evaluate(eval_dataset=tokenized_val)
print("Validation Set Results:", val_results)

test_results = trainer.evaluate(eval_dataset=tokenized_test)
print("Test Set Results:", test_results)


<ipython-input-31-3962e58e261b>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.460388,0.218750,0.221034,0.390278,0.218750
2,No log,1.471588,0.218750,0.221034,0.390278,0.218750
3,No log,1.477676,0.218750,0.221034,0.390278,0.218750
4,No log,1.486465,0.218750,0.221034,0.390278,0.218750
5,No log,1.483429,0.218750,0.185096,0.175000,0.218750
6,No log,1.514511,0.187500,0.159799,0.156250,0.187500
7,1.113000,1.532738,0.187500,0.159799,0.156250,0.187500
8,1.113000,1.569046,0.187500,0.159799,0.156250,0.187500
9,1.113000,1.564423,0.218750,0.216890,0.377083,0.218750
10,1.113000,1.586731,0.218750,0.216890,0.377083,0.218750


Validation Set Results: {'eval_loss': 1.46038818359375, 'eval_accuracy': 0.21875, 'eval_f1': 0.22103365384615387, 'eval_precision': 0.3902777777777778, 'eval_recall': 0.21875, 'eval_runtime': 0.3262, 'eval_samples_per_second': 98.102, 'eval_steps_per_second': 6.131, 'epoch': 10.0}
Test Set Results: {'eval_loss': 1.4139293432235718, 'eval_accuracy': 0.30303030303030304, 'eval_f1': 0.2680926916221034, 'eval_precision': 0.34595959595959597, 'eval_recall': 0.30303030303030304, 'eval_runtime': 0.354, 'eval_samples_per_second': 93.233, 'eval_steps_per_second': 8.476, 'epoch': 10.0}
